In [5]:
import ee, geemap
ee.Initialize()

In [2]:
def Harmonize_data(aoi_name: str, return_as_list: bool = False) -> ee.Image or list:
    """
    Retrieves land cover data for a specified area of interest (AOI).

    Args:
        aoi_name (str): Name of the AOI (e.g., 'Carpathians', 'Alps').
        return_as_list (bool): If True, returns a list of images; if False, returns a composite image.

    Returns:
        ee.Image or list: Composite image or list of individual images containing land cover data.
    """
    # Define AOI
    aoi = (ee.FeatureCollection('projects/ee-simonopravil/assets/LULC/Alps_Carph')
           .filter(ee.Filter.eq('m_range', aoi_name)))

    # Dataset configurations with remapping rules
    datasets = {
        'elc': {'path': 'projects/ee-simonopravil/assets/LULC/ELC', 'bands': ['b1'], 'remap': None},
        'clc': {'path': 'projects/ee-simonopravil/assets/LULC/CLC',
                'remap': ([1, 7, 2, 3, 4, 5, 6, 9, 10, 11], [1, 2, 3, 3, 3, 4, 5, 6, 7, 9])},
        'glc': {'path': 'projects/ee-simonopravil/assets/LULC/GLC',
                'remap': ([62, 73, 75, 82, 83, 103, 102, 121, 162, 105, 106, 123], [1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9])},
        'dw': {'path': 'projects/ee-simonopravil/assets/LULC/DW',
               'remap': ([6, 4, 1, 5, 2, 7, 0, 3, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9])},
        'esri': {'path': 'projects/ee-simonopravil/assets/LULC/ESRI',
                 'remap': ([7, 5, 2, 6, 3, 8, 1, 4, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9])},
        'esa': {'path': 'projects/ee-simonopravil/assets/LULC/ESA',
                'remap': ([50, 40, 10, 20, 30, 60, 100, 80, 90, 95, 70], [1, 2, 3, 4, 5, 6, 6, 7, 8, 8, 9])}
    }

    images = []
    for name, config in datasets.items():
        image = ee.Image(config['path'])
        band_name = 'label' if return_as_list else name

        if config['remap']:
            image = (image.remap(*config['remap'], 0)
                     .select(['remapped'], [band_name]))
        else:
            image = image.select(config['bands'], [band_name])

        image = (image.clip(aoi)
                 .toInt()
                 .set({'name': name}))
        images.append(image)

    return images if return_as_list else ee.Image(images)

In [8]:
ds = ee.ImageCollection(Harmonize_data('Alps', True))
ds